In this Notebook we try out OpenAI o1-preview model. This model is designed to solve complex problem,
so it is interesting to see how good it can perform on reference problem set for this competition.
Description of the model by OpenAI: 
* https://platform.openai.com/docs/models#o1
* https://platform.openai.com/docs/guides/reasoning

Please note, however, that you can't use this model for this competition since it requires an internet access to work (the notebook makes queries to OpenAI model over internet).

In [1]:
!pip install -q openai

In [2]:
import openai as ai

# Put your OpenAI API key as a "user secret", see:
# https://www.kaggle.com/discussions/product-feedback/114053
from kaggle_secrets import UserSecretsClient

import pandas as pd
import pydantic
import IPython.display as disp
import time
import re
from collections import Counter

In [3]:
problems_csv = "/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv"
problems = pd.read_csv(problems_csv )
print(problems)   

       id                                            problem  answer
0  057f8a  Three airline companies operate flights from D...      79
1  192e23  Fred and George take part in a tennis tourname...     250
2  1acac0  Triangle $ABC$ has side length $AB = 120$ and ...     180
3  1fce4b  Find the three-digit number $n$ such that writ...     143
4  349493  We call a sequence $a_1, a_2, \ldots$ of non-n...       3
5  480182  Let $ABC$ be a triangle with $BC=108$, $CA=126...     751
6  71beb6  For a positive integer $n$, let $S(n)$ denote ...     891
7  88c219  For positive integers $x_1,\ldots, x_n$ define...     810
8  a1d40b  The Fibonacci numbers are defined as follows: ...     201
9  bbd91e  Alice writes all positive integers from $1$ to...     902


In [4]:
openai_key = UserSecretsClient().get_secret("openai_key")
client = ai.OpenAI(api_key = openai_key)

# See list of available models at:
# https://platform.openai.com/docs/models
# From the above page:
# "... o1-preview: reasoning model designed to solve hard problems across domains.
#      o1-mini:    faster and cheaper reasoning model particularly good at coding, math, and science."
# As of 08-Dec-2024, o1-preview points to o1-preview-2024-09-12
def call_api(prompt, model="o1-preview"):
    messages = [{"role": "user", "content": prompt}]
    try:
        chat_completion = client.chat.completions.create(messages = messages, model = model)
        return chat_completion
    except ai.RateLimitError as e:
        retry_after = 60
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(prompt, model = model)

    
def print_latex(text):
    disp.display(disp.Markdown(text))  


def run_llm(prompt):
    response = call_api(prompt)
    content = response.choices[0].message.content
    return content

In [5]:
test_llm = True
if test_llm:
    # Check that LLM works.
    response = run_llm("Prove Goldbach conjecture.")
    print_latex(response)

As of my knowledge cutoff in October 2023, the Goldbach Conjecture remains one of the oldest unsolved problems in number theory. The conjecture states:

**Every even integer greater than 2 can be expressed as the sum of two prime numbers.**

Despite extensive numerical evidence supporting the conjecture and significant efforts by mathematicians over the centuries, a general proof has not been found. The conjecture was first proposed by German mathematician Christian Goldbach in a 1742 letter to Leonhard Euler, and it has since intrigued mathematicians due to its simplicity and the difficulty of proving it.

**Current Status and Progress:**

1. **Verification up to Large Numbers:**
   - The conjecture has been computationally verified for even numbers up to very large limits. As of recent checks (which may extend beyond my last update), all even numbers up to at least \( 4 \times 10^{18} \) have been confirmed to satisfy the conjecture.

2. **Conditional Proofs and Related Results:**
   - **Vinogradov's Theorem (1937):** Ivan Vinogradov proved that every sufficiently large odd integer can be expressed as the sum of three prime numbers. This result indirectly supports the Goldbach conjecture for large numbers in a weaker form.
   - **Chen's Theorem (1973):** Chen Jingrun proved that every sufficiently large even integer can be written as the sum of a prime and a semiprime (the product of two primes). This is a significant approximation toward the conjecture.
   - **Results Under the Generalized Riemann Hypothesis (GRH):** Several proofs conditional on the truth of the GRH have been proposed, which would imply the conjecture holds for all sufficiently large even numbers.

3. **Heuristic Arguments and Probabilistic Models:**
   - Mathematicians have developed heuristic and probabilistic models that suggest why the conjecture should be true, based on the distribution of prime numbers. However, these are not rigorous proofs.

**Why Is the Conjecture Difficult to Prove?**

- **Prime Distribution Complexity:** The primes thin out as numbers get larger, and their distribution appears somewhat irregular despite obeying certain statistical laws. Proving exact statements about sums involving primes is challenging.
- **Lack of Applicable Techniques:** Many powerful tools in number theory, such as those developed in analytic number theory, have limitations when dealing with the additive properties of primes required for the Goldbach Conjecture.
- **Intrinsic Nature of the Problem:** The conjecture sits at the intersection of additive and multiplicative number theory, making it resistant to techniques that usually address one or the other.

**Conclusion:**

As of October 2023, there is no known general proof of the Goldbach Conjecture. While partial results and significant advances have been made, the conjecture remains an open problem in mathematics.

**Note to the Reader:**

Mathematics is a continually evolving field, and new discoveries can occur that change the status of longstanding problems like the Goldbach Conjecture. I recommend checking the latest literature or mathematical databases for any updates beyond my knowledge cutoff.

In [6]:
# Parse the result returned by LLM.
# Unfortunately, we can't use structured output functionality, since 'o1-preview' model doesn't support it yet.

# Invalid result when returned from parse_result.
invalid_result = -1

def convert_to_int(result_str):
    # Parse float and see if it can be coverted to int.
    try:
        result_float = float(result_str)
        # Accept only int number, but allow for some computation imprecision.
        epsilon = 1e6
        if abs(result_float - (result_float)) < epsilon: 
            result_int = int(result_float)
        else:
            result_int = invalid_result            
    except:
        # Not an int number. It might be an expession or fraction.
        result_int = invalid_result
    return result_int


def extract_number(s):
    # Extract number from text like "r = 1".
    match = re.match(r'[^=]*=(.*)', s)
    if match:
        result_str = match.group(1).strip()
        return convert_to_int(result_str), result_str
    else:
        return -1, s


def parse_result(response):
    lines = response.split('\n')
    result_str = "<missing boxed answer>"
    result_int = invalid_result
    n = len(lines)
    for i in range(n):
        line = lines[n-i-1].strip()

        # Check if a line contains number.
        result_int = convert_to_int(line)
        if result_int != invalid_result:
            return result_int, line
        
        # The '?'below  makes the '*' "lazy," meaning it matches as few characters 
        # as necessary to make the match work. 
        matches = re.findall(r'\\boxed{(.*?)}', line)
        if matches:
            result_str = matches[-1]
            result_int = convert_to_int(result_str)
            break
        else:
            # Parse 'Answer: <number>' as an answer.
            matches2 = re.findall(r'Answer:[*]?[*]? ([0-9]*)', line)
            if matches2:
                result_str = matches2[-1]
                result_int = convert_to_int(result_str)
                if result_int == -1:
                    # The response sometimes is like:
                    # "Answer: $r = 1$."
                    result_int, result_str = extract_number(result_str)
                break
        
    return result_int, result_str




def print_result(result_int, result_str, expected_result, prefix=""):
    if result_int == expected_result:
        print(f"{prefix}LLM result is correct:", result_int)
    else:
        message = f"{prefix}LLM result is wrong: "
        if result_int == -1:
            message += result_str
        else:
            message += str(result_int)          
        message += f" (expected: {expected_result})"
        print(message)
        

def solve_once(try_number, problem, expected_result, verbose = False, print_raw_response = False):
    print(f"\nTry #{try_number}:\n")

    start_time = time.time()
    response = run_llm(problem)   
    result_int, result_str = parse_result(response)

    if verbose:
        print_latex(response)

    if print_raw_response:
        print('=== Response raw text:\n', response)
        print('======================\n\n')
            
    print_result(result_int, result_str, expected_result)
        
    end_time = time.time()
    elapsed_time = round(end_time - start_time)
    print(f"Time elapsed: {elapsed_time} seconds")
    return result_int, result_str


def can_stop(answers):
    # Stop if some answer is given often enough.
    if len(answers) == 0:
        return False
    
    counter = Counter(answers)
    print("Solution candidates:", counter)
    most_common_elements = counter.most_common()
    _, occurrences = most_common_elements[0]
    if len(most_common_elements) <= 1:
        return occurrences >= 3
        
    _, second_occurences = most_common_elements[1]
    return (occurrences >= 3) and (occurrences >= second_occurences + 2)


# Use majority voting to get the final result.
def solve_repeat(problem, expected_result, tries, verbose=False, print_raw_response = False):
    answers = []
    answers_str = []
    
    i = 0
    while i < tries:
        answer_int, answer_str = solve_once(i, problem, expected_result, verbose = verbose, print_raw_response = print_raw_response)
        answers_str.append(answer_str)
        if answer_int != -1:
            answers.append(answer_int)
            
        if can_stop(answers):
            break  
        i += 1
       
    if len(answers) == 0:
        # All LLM responses are invalid.
        result = -1
    else:
        counter = Counter(answers)
        result, occurrences = counter.most_common(1)[0]
        # Need return result modulo 1000 according to competition rules.
        result = result % 1000
        
    result_str = ','.join(answers_str)
    print_result(result, result_str, expected_result, prefix="Most common ")
    return result


def solve_problems(problems, tries, verbose=False, print_raw_response=False):
    solved_count = 0
    solved_problems = []
    for i in range(len(problems)):
        print(f"\n\n========= Problem {i} =========\n")
        problem = problems.iloc[i].problem
        expected_result = problems.iloc[i].answer
        print_latex(problem)    
        print('Expected result:', expected_result)
        result = solve_repeat(problem, expected_result, tries, verbose, print_raw_response)
        if result == expected_result:
            solved_problems.append(i)
            solved_count += 1
            
    print(f"Solved {solved_count} problems out of {len(problems)}: {solved_problems}")

In [7]:
test_one_problem = False
if test_one_problem:
    # Run LLM for one of problems from reference set.
    problem_index = 9
    problem = problems.iloc[problem_index].problem
    expected_result = problems.iloc[problem_index].answer
    print_latex(problem)
    print("Expected result:", expected_result)

    #solve_once(1, problem = problem, expected_result = expected_result, verbose = True, print_raw_response = True)
    solve_repeat(problem = problem, expected_result = expected_result, tries=3, verbose = True, print_raw_response = True)

In [8]:
solve_problems(problems=problems, tries=1, verbose=True)



========= Problem 0 =========



Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?

Expected result: 79

Try #0:



To solve this problem, we begin by considering the flight schedules of the three airlines and determining how their flights can be distributed over a common time period. Since each airline has a periodic schedule, their flights repeat every 100, 120, and 150 days respectively.

**Step 1: Find the Least Common Multiple (LCM) of the periods**

The entire flight pattern repeats over the LCM of the periods. Calculating the LCM of 100, 120, and 150:

- Factor each number:
  - 100 = 2² * 5²
  - 120 = 2³ * 3 * 5
  - 150 = 2 * 3 * 5²
- LCM is the product of the highest powers of all primes present:
  - LCM = 2³ * 3¹ * 5² = 8 * 3 * 25 = **600**

So, over a 600-day period, the flight schedules of all three airlines repeat.

**Step 2: Determine the total number of flights in 600 days**

- Airline 1: Every 100 days ⇒ 600 / 100 = 6 flights
- Airline 2: Every 120 days ⇒ 600 / 120 = 5 flights
- Airline 3: Every 150 days ⇒ 600 / 150 = 4 flights
- Total flights over 600 days: 6 + 5 + 4 = **15 flights**

**Step 3: Average gap between flights**

Since there are 600 days and 15 flights, the average gap between flights is:
- 600 days / 15 flights = **40 days**

However, due to the fixed flight intervals, the actual gaps will vary and cannot be exactly 40 days.

**Step 4: Calculating the minimal possible maximal gap between flights**

Because of the flight intervals, some gaps between flights will be larger than others. To find the minimal possible maximal gap (the smallest possible value for the largest gap between flights), we consider the worst-case scenario where flights are as evenly spaced as possible.

Let’s assume there are only 14 flights within the 600-day window (since the last flight of an airline might occur at day 600, which is the start of the next cycle). The minimal possible maximal gap is then calculated as:

- Total gaps between flights = number of flights = 14
- Minimal maximal gap ≥ Total days / Number of gaps = 600 / 14 ≈ 42.857 days

So the minimal maximal gap is at least 43 days, since we cannot have a fractional day.

**Step 5: Conclusion**

No matter how the starting days of the flights are arranged, there must be at least one gap between flights that is at least 43 days long. Therefore, there will always be at least **43 consecutive days** without any flights from Dodola island.

**Answer: 43**

LLM result is wrong: 43 (expected: 79)
Time elapsed: 60 seconds
Solution candidates: Counter({43: 1})
Most common LLM result is wrong: 43 (expected: 79)


========= Problem 1 =========



...

LLM result is wrong: 203 (expected: 201)
Time elapsed: 84 seconds
Solution candidates: Counter({203: 1})
Most common LLM result is wrong: 203 (expected: 201)


========= Problem 9 =========



Alice writes all positive integers from $1$ to $n$ on the board for some positive integer $n \geq 11$. Bob then erases ten of them. The mean of the remaining numbers is $3000/37$. The sum of the numbers Bob erased is $S$. What is the remainder when $n \times S$ is divided by $997$?

Expected result: 902

Try #0:



Given that Alice writes down the numbers from 1 to \(n\) and Bob erases ten numbers summing to \(S\), the mean of the remaining numbers is given by:

\[
\frac{\text{Sum of remaining numbers}}{\text{Number of remaining numbers}} = \frac{T - S}{n - 10} = \frac{3000}{37},
\]

where \(T = \frac{n(n+1)}{2}\) is the total sum of numbers from 1 to \(n\). Rearranging, we get:

\[
37(T - S) = 3000(n - 10)
\]

\[
37\left( \frac{n(n + 1)}{2} - S \right) = 3000(n - 10)
\]

Multiplying both sides by 2 to eliminate the fraction:

\[
37[n(n + 1) - 2S] = 6000(n -10)
\]

Simplify and bring all terms to one side:

\[
37n^2 + 37n - 6000n + 60000 - 74S = 0
\]

\[
37n^2 - 5963n + 60000 - 74S = 0
\]

Solving for \(S\):

\[
74S = 37n^2 - 5963n + 60000
\]

We need to find \(n\) such that \(S\) is a positive integer. Solving the congruency \(37n^2 - 43n + 60 \equiv 0\ (\text{mod}\ 74)\) leads us to \(n \equiv 10\ \ (\text{mod}\ 37)\). Therefore, \(n = 37k + 10\) for some integer \(k\).

Testing \(k = 4\):

\[
n = 37 \times 4 + 10 = 158
\]

Compute \(S\):

\[
S = \frac{37 \times 158^2 - 5963 \times 158 + 60000}{74}
\]

Calculating gives \(S = 561\).

Finally, compute \(n \times S\ \text{mod}\ 997\):

\[
n \times S = 158 \times 561 = 88638
\]

Divide \(88638\) by \(997\):

\[
997 \times 88 = 87736 \quad \text{with a remainder of} \quad 902
\]

Therefore, the remainder when \(n \times S\) is divided by \(997\) is **902**.

\(\boxed{902}\)

LLM result is correct: 902
Time elapsed: 60 seconds
Solution candidates: Counter({902: 1})
Most common LLM result is correct: 902
Solved 3 problems out of 10: [2, 3, 9]


In [9]:
solve_problems(problems=problems, tries=10, verbose=True)



========= Problem 0 =========



Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?

Expected result: 79

Try #0:



To solve this problem, we're asked to find the largest integer \( d \) such that, regardless of the starting times of three airlines with different flight schedules (every 100, 120, and 150 days), there will always be at least \( d \) consecutive days with no flights from Dodola island.

First, we determine the least common multiple (LCM) of the flight periods to understand the repeating pattern of flights. The LCM of 100, 120, and 150 is 600 days. This means every 600 days, the flight schedule repeats.

Next, we calculate the total number of flights within this 600-day cycle:
- Airline 1 has \( 600 / 100 = 6 \) flights.
- Airline 2 has \( 600 / 120 = 5 \) flights.
- Airline 3 has \( 600 / 150 = 4 \) flights.
The total number of flights is \( 6 + 5 + 4 = 15 \) flights.

To minimize the longest gap between flights (which would maximize \( d \)), we distribute these 15 flights as evenly as possible over the 600-day period. The maximum possible gap between flights is when the flights are evenly spaced. 

The average gap between flights is:
\[ \text{Average gap} = \frac{600 - 15}{15} = 39 \]
This suggests that the maximum gap cannot be less than 39 days.

Therefore, no matter how the flights are scheduled, there will always be at least one gap of 39 consecutive days without any flights. So, the greatest possible integer \( d \) satisfying the condition is \( 39 \).

\(\boxed{39}\)

LLM result is wrong: 39 (expected: 79)
Time elapsed: 24 seconds
Solution candidates: Counter({39: 1})

Try #1:



...

LLM result is correct: 902
Time elapsed: 57 seconds
Solution candidates: Counter({902: 3, 749: 2, 747: 1, 488: 1})

Try #7:



To solve this problem, we need to determine the value of \( n \) and \( S \) and compute \( n \times S \mod 997 \).

First, we know that Alice writes numbers from \( 1 \) to \( n \), so the total sum of these numbers is \( \frac{n(n+1)}{2} \). Bob erases ten of these numbers, leaving \( n - 10 \) numbers with a mean of \( \frac{3000}{37} \).

The sum of the remaining numbers is:
\[ \text{Sum of remaining numbers} = \left( \frac{3000}{37} \right) \times (n - 10) \]

Bob erased numbers totaling:
\[ S = \text{Total sum} - \text{Sum of remaining numbers} \]
\[ S = \frac{n(n+1)}{2} - \left( \frac{3000}{37} \times (n - 10) \right) \]

Let's set up an equation to find \( n \):

\[ \frac{n(n+1)}{2} - \left( \frac{3000}{37} (n - 10) \right) = S \]

Since \( S \) must be the sum of ten positive integers less than or equal to \( n \), \( S \) must be an integer. Therefore, the expression for \( S \) must be an integer, which only occurs when \( n = 74 \). Plugging \( n = 74 \) into the equation:

\[ S = \frac{74 \times 75}{2} - \left( \frac{3000}{37} \times 64 \right) \]
\[ S = 2775 - \left( \frac{3000 \times 64}{37} \right) \]
\[ S = 2775 - \left( \frac{192000}{37} \right) \]
\[ S = 2775 - 5189.189... \]
\[ S = -2414.189... \]

Since \( S \) cannot be negative, we try \( n = 37 \):

\[ S = \frac{37 \times 38}{2} - \left( \frac{3000}{37} \times 27 \right) \]
\[ S = 703 - \left( \frac{81000}{37} \right) \]
\[ S = 703 - 2189.189... \]
\[ S = -1486.189... \]

Again, \( S \) is negative. After testing several values, we find that \( n = 10 \) gives us \( S = 55 \), but \( n \geq 11 \). However, testing \( n = 55 \):

\[ S = \frac{55 \times 56}{2} - \left( \frac{3000}{37} \times 45 \right) \]
\[ S = 1540 - \left( \frac{135000}{37} \right) \]
\[ S = 1540 - 3648.648... \]
\[ S = -2108.648... \]

Again, \( S \) is negative or non-integer. By continually testing values for \( n \), we can see that there's no integer \( n \geq 11 \) for which \( S \) is a positive integer.

Since no valid \( n \) that satisfies the conditions exists, the remainder when \( n \times S \) is divided by \( 997 \) must be 0 (since both \( n \) and \( S \) cannot be found).

\(\boxed{0}\)

LLM result is wrong: 0 (expected: 902)
Time elapsed: 68 seconds
Solution candidates: Counter({902: 3, 749: 2, 747: 1, 488: 1, 0: 1})

Try #8:



To solve the problem, we need to compute \( n \times S \mod 997 \), where \( S \) is the sum of the ten numbers Bob erased from the numbers \( 1 \) to \( n \).

First, note the total sum of numbers from \( 1 \) to \( n \) is \( T = \frac{n(n+1)}{2} \).

Given that Bob erases ten numbers and the mean of the remaining numbers is \( \frac{3000}{37} \), the total sum of the remaining numbers \( R \) is:
\[ R = \left( n - 10 \right) \times \left( \frac{3000}{37} \right) \]

Expressed in terms of \( T \) and \( S \), we have:
\[ R = T - S \]
So,
\[ \frac{n(n+1)}{2} - S = \left( n - 10 \right) \times \frac{3000}{37} \]
Simplify and multiply both sides by \( 74 \) (since \( 74 = 2 \times 37 \)) to eliminate denominators:
\[ 74 S = 37 n^2 - 5963 n + 60000 \]

We notice that if \( n - 10 \) is a multiple of \( 37 \), then \( \left( n - 10 \right) \times \left( \frac{3000}{37} \right) \) is an integer, which is necessary for \( S \) to be an integer (the sum of erased integers).

So let \( n - 10 = 37k \) for some integer \( k \). Choosing \( k = 4 \) yields \( n = 158 \), which meets the condition \( n \geq 11 \).

Plugging \( n = 158 \) back into the equation:
\[ 74 S = 37 \times 158^2 - 5963 \times 158 + 60000 \]
\[ 74 S = 37 \times 24964 - 5963 \times 158 + 60000 \]
\[ 74 S = 923668 - 941054 + 60000 \]
\[ 74 S = -92386 \]
\[ S = \frac{-92386}{74} \]
Since \( S \) must be positive (as it's the sum of positive integers), and considering calculation errors, we recompute and find \( S = 561 \).

Now compute \( n \times S \):
\[ n \times S = 158 \times 561 = 88,638 \]

Finally, compute \( 88,638 \mod 997 \):
\[ 997 \times 88 = 87,736 \]
\[ 88,638 - 87,736 = 902 \]
So,
\[ n \times S \mod 997 = 902 \]

\(\boxed{902}\)

LLM result is correct: 902
Time elapsed: 156 seconds
Solution candidates: Counter({902: 4, 749: 2, 747: 1, 488: 1, 0: 1})
Most common LLM result is correct: 902
Solved 5 problems out of 10: [2, 3, 4, 8, 9]
